## Stocks from nasdaq web for single stock

In [ ]:
'''load packages'''
import requests
from bs4 import BeautifulSoup
 
site = 'http://www.nasdaq.com/symbol/nflx/news-headlines'

In [2]:
'''scrape the html of the site'''
html = requests.get(site).content
 
'''convert html to BeautifulSoup object'''
soup = BeautifulSoup(html , 'lxml')
 
'''get list of all links on webpage'''
links = soup.find_all('a')
 
urls = [link.get('href') for link in links]
urls = [url for url in urls if url is not None]
 
len(urls)
'''226'''

'226'

In [7]:
'''Filter the list of urls to just the news articles'''
news_urls = [url for url in urls if '/article/' in url]
 
len(news_urls)
'''6'''

'6'

In [9]:
'''Package what we did above into a function'''
def scrape_news_text(news_url):
 
    news_html = requests.get(news_url).content
 
    '''convert html to BeautifulSoup object'''
    news_soup = BeautifulSoup(news_html , 'lxml')
 
    paragraphs = [par.text for par in news_soup.find_all('p')]
    news_text = '\n'.join(paragraphs)
 
    return news_text

In [13]:
'''Generalized function to get all news-related articles from a Nasdaq webpage'''
def get_news_urls(links_site):
    '''scrape the html of the site'''
    resp = requests.get(links_site)
 
    if not resp.ok:
        return None
 
    html = resp.content
 
    '''convert html to BeautifulSoup object'''
    soup = BeautifulSoup(html , 'lxml')
 
    '''get list of all links on webpage'''
    links = soup.find_all('a')
 
    urls = [link.get('href') for link in links]
    urls = [url for url in urls if url is not None]
 
    '''Filter the list of urls to just the news articles'''
    news_urls = [url for url in urls if '/article/' in url]
 
    return news_urls

In [11]:
def scrape_all_articles(ticker , upper_page_limit = 5):
 
    landing_site = 'http://www.nasdaq.com/symbol/' + ticker + '/news-headlines'
 
    all_news_urls = get_news_urls(landing_site)
 
    current_urls_list = all_news_urls.copy()
 
    index = 2
 
    '''Loop through each sequential page, scraping the links from each'''
    while (current_urls_list is not None) and (current_urls_list != []) and \
        (index <= upper_page_limit):
 
        '''Construct URL for page in loop based off index'''
        current_site = landing_site + '?page=' + str(index)
        current_urls_list = get_news_urls(current_site)
 
        '''Append current webpage's list of urls to all_news_urls'''
        all_news_urls = all_news_urls + current_urls_list
 
        index = index + 1
 
    all_news_urls = list(set(all_news_urls))
 
    '''Now, we have a list of urls, we need to actually scrape the text'''
    all_articles = [scrape_news_text(news_url) for news_url in all_news_urls]
 
    return all_articles

In [17]:
nflx_articles = scrape_all_articles('nflx' , 5)
 
print(nflx_articles[0])

['Join the Nasdaq Community today and get free, instant access to portfolios, stock ratings, real-time alerts, and more!\n\nInvestorPlace - Stock Market News, Stock Advice & Trading Tips\nEarlier this year,\xa0 IQIYI \xa0(NASDAQ: IQ ) stock was the mother of all momentum stocks, yet the year-to-date scoreboard shows that the stock has done nothing overall. It is important to note that the dip in IQ stock is not all from intrinsic problems.\nAll Chinese stocks have been under severe pressure. In fact, IQ stock has done better than the group. The\xa0 iShares China Large-Cap ETF \xa0(NYSEARCA: FXI ) is down 17% year-to-date; Baidu (NASDAQ: BIDU ) and Alibaba (NYSE: BABA ) are down over 25%.\nIQIYI stock is a relatively new entrant to the stock market. It came out of the gate earlier this year like a rocket rallying to $46 per share. But the 180% rally came crashing back down to earth after its June highs. Now it is struggling to recover $16 per share.\xa0So for those who missed the IPO op

In [15]:
get_news_urls('http://www.nasdaq.com/symbol/nflx/news-headlines')

['https://www.nasdaq.com/article/disruptor-alert-these-4-companies-are-changing-the-financial-industry-cm1075811',
 'https://www.nasdaq.com/article/disruptor-alert-these-4-companies-are-changing-the-financial-industry-cm1075811',
 'https://www.nasdaq.com/article/how-data-is-making-the-us-stock-market-open-for-all-investors-cm998274',
 'https://www.nasdaq.com/article/how-data-is-making-the-us-stock-market-open-for-all-investors-cm998274',
 'https://www.nasdaq.com/article/use-post-holiday-sales-to-treat-yourself-responsibly-cm1073992',
 'https://www.nasdaq.com/article/use-post-holiday-sales-to-treat-yourself-responsibly-cm1073992',
 'https://www.nasdaq.com/article/netflixs-bird-box-was-a-blockbuster-hit-cm1076139',
 'https://www.nasdaq.com/article/notable-etf-inflow-detected-itot-nflx-adbe-avgo-cm1076087',
 'https://www.nasdaq.com/article/sp-500-movers-lnt-nflx-cm1076048',
 'https://www.nasdaq.com/article/spotify-stock-is-due-for-a-big-comeback-says-analyst-cm1076031',
 'https://www.nasd

## Article from Yahoo Finance for all Stocks

In [26]:
import pandas as pd
 
dow_info = pd.read_html('https://finance.yahoo.com/quote/%5EDJI/components?p=%5EDJI')[0]
 
dow_tickers = dow_info.Symbol.tolist()

from yahoo_fin import stock_info as si
# return list of Dow tickers
dow_tickers = si.tickers_dow()

In [27]:
dow_articles = {ticker : scrape_all_articles(ticker , 1) for ticker in dow_tickers}

In [28]:
dow_articles['AAPL']

['Join the Nasdaq Community today and get free, instant access to portfolios, stock ratings, real-time alerts, and more!\n\nInvestorPlace - Stock Market News, Stock Advice & Trading Tips\nApple (NASDAQ: AAPL ) got a lump of coal in its stocking this holiday season, courtesy of Netflix (NASDAQ: NFLX ). In-app subscriptions to the streaming video service have been a significant contributor to Apple Services revenue, thanks to the chunk of every payment that goes to Apple when someone subscribes through an iOS app. However, a few days after Christmas, Netflix confirmed that it no longer allows sign-up and payment through the iOS app, requiring users to instead use its website.\nThat\'s a lot less convenient for new Netflix users but more importantly from AAPL\'s perspective, the move eliminates its cut from the most lucrative app on the App Store.\nThrough the summer, Netflix fired a warning shot at AAPL . The company updated its iOS app, preventing users in some countries from paying for